In [ ]:
%load_ext autoreload
%autoreload 2

import os
from os.path import join as pjoin
import pandas as pd
import seaborn as sns; print(sns.__version__)
from datetime import datetime
import pingouin as pg
from bmp_config import path_data
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from bmp_behav_proc import *

import numpy as np

ps_2nice = dict( zip(['pre','pert','washout','rnd'], 
        ['No perturbation','Perturbation','Washout','Random']) )


target_angs = (np.array([157.5, 112.5, 67.5, 22.5]) + 90) * (np.pi / 180)
target_angs / np.pi * 180


In [ ]:
fnf = pjoin(path_data,'df_all_multi_tsz__.pkl.zip')
#fnf = '/home/demitau/ownCloud/Current/merr_data/df_all_multi_tsz.pkl.zip'
print(fnf)
print( str(datetime.fromtimestamp(os.stat(fnf).st_mtime)))
df_all_multi_tsz = pd.read_pickle(fnf)
#dfc_multi_tsz = pd.   read_pickle(pjoin(path_data,'df_all_multi_tsz_trunc_q=0.05.pkl.zip'))
print(df_all_multi_tsz['trial_group_col_calc'].unique())
print(df_all_multi_tsz['retention_factor_s'].unique())
# df = df_all_multi_tsz.query('trial_shift_size == 1 and trial_group_col_calc == "trialwe" '
#                             ' and retention_factor_s == "1.000"').copy()
df_all_multi_tsz_orig = df_all_multi_tsz

In [ ]:
df = df_all_multi_tsz.query('trial_shift_size == 1 and trial_group_col_calc == "trialwe" '
                           ' and retention_factor_s == "0.924"').copy().sort_values(['subject','trials'])
df_orig = df

#%debug
# define dfall here
df,dfall,ES_thr,envv,pert = addBehavCols2(df);

df_wthr = df.copy()

In [ ]:
assert df_orig.groupby('subject').size().min() == 768

### within target

#### Fig2

In [ ]:
from figure.plots import relplot_multi
df2 = df_all_multi_tsz.\
    query('trial_shift_size == 1 and '
          'trial_group_col_calc.isin(["trialwtgt_we","trialwe"]) '
            ' and retention_factor_s == "1.000"').copy()

dfni = df2.loc[~np.isinf(df2['err_sens'])]
dfni_g = dfni.query('err_sens.abs() <= @ES_thr')
nremoved_pooled = len(dfni) - len(dfni_g)

sz = dfni.groupby(['subject'],observed=True).size()
sz_g = dfni_g.groupby(['subject'],observed=True).size()
mpct = ((sz - sz_g) / sz).mean() * 100
print(f'Mean percentage of removed trials = {mpct:.2f}%, '
      f'pooled = {nremoved_pooled / len(dfni) * 100:.2f}%')

dfall2 = truncateDf(dfni_g, 'err_sens', q=0.0, infnan_handling='discard', 
                   cols_uniqify = ['subject','env','trial_group_col_calc'])

In [ ]:
fg,df= relplot_multi(data=dfall2,
              ys=[['error_pscadj'],
                 ['err_sens']], x='trials',
                height=4,aspect=2, kind='line',
                     col = 'trial_group_col_calc',
                    estimator='mean', errorbar='sd')
fg.refline(y=0)
#r[0].refline(x=526)

In [ ]:
df_witgt = df_all_multi_tsz.query('trial_shift_size == 1 and trial_group_col_calc == "trialwtgt_we" '
                           ' and retention_factor_s == "0.924"').copy().sort_values(['subject','trials'])
df_witgt,dfall_witgt,ES_thr_witgt,_,_ = addBehavCols2(df_witgt.copy());

In [ ]:
from figure.plots import relplot_multi
df2 = df_all_multi_tsz.\
    query('trial_shift_size == 1 and '
          'trial_group_col_calc.isin(["trialwtgt_we","trialwe"]) '
            ' and retention_factor_s == "1.000"').copy()

dfni = df2.loc[~np.isinf(df2['err_sens'])]
dfni_g = dfni.query('err_sens.abs() <= @ES_thr')
nremoved_pooled = len(dfni) - len(dfni_g)

sz = dfni.groupby(['subject'],observed=True).size()
sz_g = dfni_g.groupby(['subject'],observed=True).size()
mpct = ((sz - sz_g) / sz).mean() * 100
print(f'Mean percentage of removed trials = {mpct:.2f}%, '
      f'pooled = {nremoved_pooled / len(dfni) * 100:.2f}%')

dfall2 = truncateDf(dfni_g, 'err_sens', q=0.0, infnan_handling='discard', 
                   cols_uniqify = ['subject','env','trial_group_col_calc'])

In [ ]:
fg,df= relplot_multi(data=dfall2,
              ys=[['error_pscadj'],
                 ['err_sens']], x='trials',
                height=4,aspect=2, kind='line',
                     col = 'trial_group_col_calc',
                    estimator='mean', errorbar='sd')
fg.refline(y=0)
#r[0].refline(x=526)
plt.show()

# ES vs prev err (supp plot)

In [ ]:
#dfc = df_wthr.copy()
dfc = dfall.copy()
assert dfc.err_sens.max() < 40

# #bins = np.linspace(-1.5,1.5,10)
# bins = np.linspace(-1.5,1.5,10)
# dfc['prev_error_bin'] = pd.cut(dfc['prev_error'], bins)
# grp = dfc.groupby( ['subj','prev_error_bin','env'] )
# dfme = grp['err_sens'].mean().reset_index()
# dfme['prev_error_mid'] = dfme['prev_error_bin'].apply(lambda x: x.mid )
# dfme

In [ ]:
from figure.plots import genStRandLegendHandles
import warnings
handles = genStRandLegendHandles(rect=False, include_labels=True)

r2d = 180 / np.pi
savefig = 1
bins = np.linspace(-1.2,1.2,15) * r2d; 
#bins = np.linspace(-1.5,1.5,15)  * r2d
#bins = np.linspace(-1.5,1.5,10) * r2d
dfc['prev_error_deg_bin'] = pd.cut(dfc['prev_error'] * r2d, bins) # important to use true error, not pscadj
grp = dfc.groupby( ['subj','prev_error_deg_bin','env','dist_rad_from_prevtgt2'] )
dfme = grp['err_sens'].mean().to_frame()
dfstd = grp['err_sens'].std().to_frame()
dfmesz = grp['err_sens'].size().to_frame()
dfme['sz'] = dfmesz['err_sens']
dfme['err_sens_std'] = dfstd['err_sens']
dfme = dfme.reset_index()
dfme['prev_error_deg_mid'] = dfme['prev_error_deg_bin'].apply(lambda x: x.mid )

nbins = len(bins)
minsz = 2  # minimum number of points we want within a bin
ylim = -0.65,1.1
df_ = dfme.query('sz >= @minsz')

def custom_axis_label(ax, data):
    env = data.env.values[0]
    d = data.dist_rad_from_prevtgt2.values[0]
    ax.set_label_text(f'Dist = {d}')
    
def set_custom_titles(grid, data):
    for ax, (col_val, data_cur_axis) in zip(grid.axes.flat, grid.facet_data()):
        d = data_cur_axis.dist_rad_from_prevtgt2.values[0]
        d = float(d) * r2d
        ax.set_title(f'Distance from\nprevious target = {d:.0f} [deg]')                

#row='env',
with warnings.catch_warnings():
    warnings.filterwarnings('ignore',category=FutureWarning)
    fg = sns.relplot(kind='line', data=df_, x='prev_error_deg_mid', y='err_sens', 
                      col='dist_rad_from_prevtgt2', hue='env',
                    height=5, hue_order=['stable','random'], palette=['tab:orange','tab:grey'],
                     col_order=['0.00', '0.79', '1.57', '2.36'],
                    legend=False, aspect=0.9)
    #fg = sns.scatterplot(data=dfc.query('env == "stable"'), x='prev_error', y='err_sens', hue='block_name')
    #fg.axhline(0,c='r',ls=':')
    for ax in fg.axes.flatten():
        ax.axhline(0,c='r',ls=':')
        ax.axvline(0,c='r',ls=':')
        ax.set_ylabel('error sensitivity')
        ax.set_xlabel('previous error [deg]')
        ax.set_ylim(ylim)
        
    set_custom_titles(fg, df_)
    
    ax.legend(handles=handles, facecolor='white',                                                                                                                                                                                                       
    loc='lower right', framealpha=0.75, fontsize='medium')  

        #ax.set_ylim(-15,15)
    #addTitleInfo(ax)
    
    plt.suptitle('ES dependence on previous error')
    plt.tight_layout()
    
    print('minsz = ',minsz)
    if savefig:
        plt.savefig(pjoin(path_fig, 'behav', f'ES_per_prev_error_binned_inctgt_minsz={minsz}_{nbins}.pdf'))
    else:
        print('Skipping fig saving')

In [ ]:
dft = df_.query('dist_rad_from_prevtgt2 == "0.00"')
#[['subj','env','err_sens','prev_error_deg_mid']]

In [ ]:
dfc.groupby( ['subj','prev_error_deg_bin','env','dist_rad_from_prevtgt2'] )

In [ ]:
myttest?

In [ ]:
dfc['prev_error_deg_mid'] = dfc['prev_error_deg_bin'].apply(lambda x: x.mid ).astype('category')

In [ ]:
def f(df):
    #df1 = df.query('env == "stable"')
    #df2 = df.query('env == "random"')
    try:
        r = myttest( df, 'env == "stable"', 'env == "random"', 'err_sens', alt='two-sided')
        #print(len(r))
    except Exception as e:
        print(e)
        r = None
    return r

dfcst = dft.query('dist_rad_from_prevtgt2 == "0.00"')
grp_ = dfcst.groupby(['prev_error_deg_mid'])
dftr = grp_.apply(f)#
dftr
#['err_sens']

In [ ]:
def f(df):
    #df1 = df.query('env == "stable"')
    #df2 = df.query('env == "random"')
    try:
        r = myttest( df, 'env == "stable"', 'env == "random"', 'err_sens', alt='two-sided')
        #print(len(r))
    except Exception as e:
        print(e)
        r = None
    return r

dfcst = dfme.query('dist_rad_from_prevtgt2 == "0.00"')
grp_ = dfcst.groupby(['subj','prev_error_deg_mid'])
dftr = grp_.apply(f)#
#['err_sens']

In [ ]:
grp_.size().max()

In [ ]:
dftr

In [ ]:
#mne.stats.permutation_cluster_test(

In [ ]:
dfcst = dfcst.copy()

In [ ]:
dfcst['subj'] = dfcst['subj'].astype('category')

In [ ]:
dfcst = dfcst.reset_index()

In [ ]:
#model = smf.mixedlm("err_sens ~ C(env) * C(prev_error_deg_mid)", dfcst, groups=dfcst["subj"])
#result = model.fit()
#result.summary()

In [ ]:
dftr[~dftr['T'].isna()]

In [ ]:
# plot standard deviation
ylim = -0.65,3
with warnings.catch_warnings():
    warnings.filterwarnings('ignore',category=FutureWarning)
    fg = sns.relplot(kind='line', data=df_, x='prev_error_deg_mid', y='err_sens_std', 
                      col='dist_rad_from_prevtgt2', hue='env',
                    height=5, hue_order=['stable','random'], palette=['tab:orange','tab:grey'],
                     col_order=['0.00', '0.79', '1.57', '2.36'],
                    legend=False, aspect=0.9)
    #fg = sns.scatterplot(data=dfc.query('env == "stable"'), x='prev_error', y='err_sens', hue='block_name')
    #fg.axhline(0,c='r',ls=':')
    for ax in fg.axes.flatten():
        ax.axhline(0,c='r',ls=':')
        ax.axvline(0,c='r',ls=':')
        ax.set_ylabel('error sensitivity standard deviation')
        ax.set_xlabel('previous error [deg]')
        ax.set_ylim(ylim)
        
    set_custom_titles(fg, df_)
    
    ax.legend(handles=handles, facecolor='white',                                                                                                                                                                                                       
    loc='lower right', framealpha=0.75, fontsize='medium')  

        #ax.set_ylim(-15,15)
    #addTitleInfo(ax)
    
    plt.suptitle('ES std dependence on previous error')
    plt.tight_layout()

# Additional stats (supp)

In [ ]:
mt = 0.754
st = 0.190
s = f'Mean movement time = {mt:.2f}s (std = {st:.2f}s)'
print(s)

In [ ]:
dfall.groupby('subject')['trial_duration'].mean().describe()

In [ ]:
dfall.groupby('subject')['movement_duration'].mean().describe()

In [ ]:
from behav_proc import comparePairs    
df_ = dfall.query('thr == "mestd*0" and env == "random"')
varn = 'err_sens'
col = 'block_name'

print('Compare different random blocks')
df_ = dfall.query('env == "random"')
varn = 'err_sens'
col = 'block_name'

ttrssig, ttrs = comparePairs(df_,varn,col)
if ttrssig is None:
    print('None')
else:
    ttrssig[statcols_toshow]
ttrs

In [ ]:
#'val1','val2',
statcols_toshow = ['varn','T','pval','val1_nice','val2_nice','alternative','pooled','ttstr', 'ttstr_nice','dof']

In [ ]:
stages = ['pre1','pert1_1','washout1_1','pert1_2','washout1_2'] + ['pre2','pert2_1','washout2_1','pert2_2','washout2_2']
ind2stage = dict(zip(np.arange(len(stages)),stages))
ind2stage[-1] = 'random'
ind2stage

def nicener(s):
    # we have to start from -1 otherwise bad
    inds = [-1] + list(range(10))
    for i in inds:
        ps = ind2stage[i]
        
        si = str(i)
        sil = len(si)
        if s.startswith(si + ' '):# or s.startswith(si + '>'):
            s = ps + s[sil:]
        elif s.endswith(' ' + si):# or s.endswith('>' + si):
            s = s[:-sil] + ps
        
    return s
ttrssig['ttstr_nice'] = ttrssig['ttstr'].apply(nicener)
ttrssig['ttstr_nice']

In [ ]:
me = dfall.groupby(['thr','subject','pert_stage']).\
    mean(numeric_only=1).reset_index()
me_pert_stage = me

for col, me in [('pert_stage',me_pert_stage)]:#[('env', me_env), ('ps_', me_ps)]:
    for qs in ['thr == "mestd*0"']:
        df_ = dfall #me.query(qs)
        print(col, qs)
        varn = 'err_sens'
        ttrssig, ttrs = comparePairs(df_,varn,col)
        ttrssig['val1_nice'] = ttrssig['val1'].apply(lambda x: ind2stage[x])
        ttrssig['val2_nice'] = ttrssig['val2'].apply(lambda x: ind2stage[x])
        ttrssig['ttstr_nice'] = ttrssig['ttstr'].apply(nicener)
        
        display(ttrssig.query('alternative != "two-sided"')\
                [statcols_toshow])

In [ ]:
display( ttrssig.query('alternative != "two-sided" and not ttstr.str.contains("-1")')[['ttstr_nice','pval']] )

In [ ]:
display( ttrssig.query('alternative != "two-sided" and not ttstr.str.contains("-1") and val1_nice.str.contains("pert") and val2_nice.str.contains("pert")')[['ttstr_nice','pval']] )
display( ttrssig.query('alternative != "two-sided" and not ttstr.str.contains("-1") and val1_nice.str.contains("wash") and val2_nice.str.contains("wash")')[['ttstr_nice','pval']] )
display( ttrssig.query('alternative != "two-sided" and not ttstr.str.contains("-1") and val1_nice.str.contains("pre") and val2_nice.str.contains("pre")')[['ttstr_nice','pval']] )
print("Last pertrubation has lower ES compared to first and third perturbations (p-values 0.035 and 0.016.")
print("First washout has lower ES compared to the second one (p-value 0.014).")

# within tgt main stats

## no savings 

In [ ]:
corrs_per_subj_me_,corrs_per_subj  = corrMean(dfall_witgt, 
                stagecol = 'pert_stage', coln='err_sens')

# show stat signif
stage_pairs = [(1,6),(3,8)]
ttrs = []
for s1,s2 in stage_pairs:    
    cps1 = corrs_per_subj.reset_index().query('pert_stage == @s1').set_index(['subject'])
    cps2 = corrs_per_subj.reset_index().query('pert_stage == @s2').set_index(['subject'])
    cps_dif = cps1['r'] - cps2['r'] 
    cps_dif= cps_dif.to_frame()        
    ttr = compare0(cps_dif, 'r', alt='two-sided')
    ttr['stage_pair'] = f'{s1}-{s2}'
    ttrs += [ttr]
ttrs = pd.concat(ttrs)
display( ttrs.query('pval <= 1e-2') )

stage_pairs_nice = {"1-6":'first and last', "3-8":'second and third'}

display(ttrs)
print('\n\nNo savings:')
for irow,row in ttrs.iterrows():
    sp = row['stage_pair']
    pv=row['pval']

    #print(sp,pv)
    print('ES during {} perturbations are not significantly different, p-value = {:.2e}.'.\
              format(stage_pairs_nice[sp],pv) )

## Fig 2 CD

In [ ]:
## plots

In [ ]:
#me.subject.nunique()

In [ ]:
from figure import renameTickLabels, palette_stabrand

thr = "mestd*0"
me = dfall_witgt.groupby(['thr','subject','env'], observed=True).\
    mean(numeric_only=1).reset_index()
me_env = me
#me.groupby(['env','thr']).size()
sns.set(font_scale=1.3)
fg = sns.catplot(data = me.query('thr == @thr'), 
                 kind='violin', y='err_sens', 
    x='env', col='thr', order=['stable','random'],
                palette = palette_stabrand)
#addTitleInfo(fg.axes.flatten()[0])
for ax in fg.axes.flatten():
    ax.axhline(y=0, c='r', ls=':'); #ax.set_ylim(-5,5)
    
from figure.mystatann import plotSigAll
ylast, ttrssig_env = plotSigAll(ax, 0.83, 0.05, ticklen=0.02,
       df=me, coln='err_sens', colpair = 'env')
    
ax.annotate('C', xy=(0, 1), xytext=(-60, 60), 
      fontsize=19, fontweight='bold', va='top', ha='left',
      xycoords='axes fraction', textcoords='offset points')
fg.set_ylabels('Error sensitivity')
ax.set_xlabel('Environment')

fign = 'Fig2C'
plt.savefig(pjoin(path_fig, 'behav',fign + '.svg'))
plt.savefig(pjoin(path_fig, 'behav',fign + '.pdf'))
plt.show()
    
###################
# non-pooled stages and env comparison
me = dfall_witgt.groupby(['thr','subject','ps2_'], observed=True).\
    mean(numeric_only=1).reset_index()
me_ps = me
#me.groupby(['ps_','thr']).size()
sns.set(font_scale=1.3)
fg = sns.catplot(data = me.query('thr == @thr'), kind='violin', y='err_sens', 
    x='ps2_', col='thr', order=['pre','pert','washout','rnd'],
                hue = 'ps2_')
for ax in fg.axes.flatten():
    ax.axhline(y=0, c='r', ls=':'); #ax.set_ylim(-5,5)
#addTitleInfo(fg.axes.flatten()[0])
#plt.gcf().add_subplot_labels(['C'])

ylast, ttrssig_ps = plotSigAll(ax, 2.05, 0.14, ticklen=0.05,
       df=me, coln='err_sens', colpair = 'ps2_', fontsize = 10)

ax.annotate('D', xy=(0, 1), xytext=(-60, 60), 
      fontsize=19, fontweight='bold', va='top', ha='left',
      xycoords='axes fraction', textcoords='offset points')

ax.set_xlabel('Experiment stage')
#ax.set_xticklabels(['No perturbation','Perturbation','Washout','Random'], 
#                   rotation=30)
ps_2nice = dict( zip(['pre','pert','washout','rnd'], 
        ['No perturbation','Perturbation','Washout','Random']) )
renameTickLabels(ax, ps_2nice, rotation=30)

fg.set_ylabels('Error sensitivity')
fign = 'Fig2D'
plt.savefig(pjoin(path_fig, 'behav',fign + '.svg'))
plt.savefig(pjoin(path_fig, 'behav',fign + '.pdf'))

#fg.set_titles()

In [ ]:
## stats

In [ ]:
# to put in results
ttrs = []
for env in me_env.env.unique():
    r = compare0(me_env.query('env == @env'),'err_sens',cols_addstat=['err_sens'])
    r['env'] = env
    ttrs += [r]
ttrs_pos = pd.concat(ttrs).query('pval <= 0.05').set_index('env')
assert len(ttrs_pos) == 2
display(ttrs_pos)

s0 = 'Averages of ES within participant. \n'
for env in me_env.env.unique():
    row = ttrs_pos.loc[env]
    s = (f"{env} ES mean = {row['err_sens_mean']:.2f} (std = {row['err_sens_std']:.2f}),"
        f" ES > 0 p-value = {row['pval']:.2e}. \n" )
    s = s[0].upper() + s[1:]
    s0 += s

display(ttrssig_env)
row = ttrssig_env.iloc[0]

s = f"Stable > random p-value={row['pval']:.2e}"
print(s0)
print(s)

In [ ]:
# to put in results
#cocoln = 'ps2_'
ttrs = []
for ps in me_ps.ps2_.unique():
    r = compare0(me_ps.query('ps2_ == @ps'),'err_sens',cols_addstat=['err_sens'])
    r['ps2_'] = ps
    ttrs += [r]
ttrs_pos = pd.concat(ttrs).query('pval <= 0.05')
assert len(ttrs_pos) == 4
display(ttrs_pos)
ttrs_pos = ttrs_pos.set_index('ps2_')

s0 = 'Averages of ES within participant: \n'
for ps in me_ps.ps2_.unique():
    row = ttrs_pos.loc[ps]
    s = (f"{ps} ES mean = {row['err_sens_mean']:.2f} (std = {row['err_sens_std']:.2f}),"
        f" ES > 0 p-value = {row['pval']:.2e}. \n" )
    s = s[0].upper() + s[1:]
    s0 += s

#display(ttrssig_ps)
#row = ttrssig_ps.iloc[0]

ttrssig_ps,_ = comparePairs(me_ps, 'err_sens', 'ps2_', alt=['greater'], 
                            paired=True, updiag=False)
display(ttrssig_ps[ttrssig_ps.columns[-5:]])
s1 = ''
for i,row in ttrssig_ps.iterrows():
    ttstr = row['ttstr']
    s = ttstr.replace(row['val2'], ps_2nice[row['val2']] ).replace(row['val1'],
                                                                   ps_2nice[row['val1']] )    
    s += f" p-value={row['pval']:.2e}; \n"
    s1 += s
#s = f"Stable > random p-value={row['pval']:.2e}"
print(s0)
print(s1)

In [ ]:
# show that dif between pert and rand is not there
psvals = ["pert","rnd"]
ttrssig,ttrssig_all = comparePairs(me_ps.query('ps2_.isin(@psvals)'),
                         'err_sens', 'ps2_', alt=['two-sided'], 
                        paired=True, updiag=True, pooled=0)
display(ttrssig_all)
assert len(ttrssig_all) == 1
row = ttrssig_all.iloc[0]
if row['pval'] > 0.05:    
    s = 'The difference between ES in '
    s += '{} and {} conditions is not significant ('.format( 
        ps_2nice[row['val2']], ps_2nice[row['val1'] ] )
    s += f" p-value={row['pval']:.2e})."
    #s1 += s
    print(s)
else:
    print('Actually there is significance')

# New stats for err

In [ ]:
warnings.filterwarnings("ignore",category=UserWarning) 

In [ ]:
ttrssig,ttrs = comparePairs(dfall,'error_pscadj','block_name')

In [ ]:
ttrssig.query('alternative != "two-sided"')[['pval','ttstr','pooled']]

In [ ]:
me = dfall.groupby(['subject','env','block_name'])['error_pscadj'].mean().to_frame().reset_index()

In [ ]:
me

In [ ]:
from behav_proc import compare0
for bn in dfall.block_name.unique():
    df_ = dfall.query('block_name == @bn')
    ttrs = compare0(df_,'error_pscadj').query('pval <= 0.05')
    print(bn)
    display(ttrs)

In [ ]:
fg = sns.catplot(data = dfall,kind='violin',y='error_pscadj',
           x='block_name', hue='env')
fg.refline(y=0)

fg = sns.catplot(data = me,kind='violin',y='error_pscadj',
           x='block_name', hue='env')
fg.refline(y=0)